<a href="https://colab.research.google.com/github/prabal4546/Fake-Review-Detection/blob/main/NLP_Assignment_3_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import libraries

In [ ]:
import pandas as pd
from IPython.display import display
import numpy as np
import os
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, GridSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from spellchecker import SpellChecker
import spacy
# from google.colab import drive
import pandas as pd
from IPython.display import display


In [ ]:
# drive.mount('/content/drive')
# df = pd.read_excel('/content/drive/MyDrive/Dataset/restaurant_reviews-v2-1.xlsx')

### Loading dataset (Please read comments to proceed)

In [ ]:
df = df = pd.read_excel('') # copy paste youe dataset PATH b/w quotes

In [ ]:
display(df)

,Restaurant,Review,Real=1/Fake=0,positive=1/negative=0,AWL,ASL,NOW,NVB,NAJ,NPV,NST,CDV,NTP,TPR
0,Tikka Shack,Great food and great atmosphere! The chicken t...,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Tikka Shack,I had heard good things about Tikka Shak so I ...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Tikka Shack,I was driving by tikka shack one day and decid...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Tikka Shack,Tikka Shack had the most modern and up-to-date...,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,India Palace\nIndian\nRestaurant,Today is the third time I've come to India Pal...,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255,Tikka Shack,I usually don't eat outside very often and Tik...,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
256,Indian Palace Restaurant,The outside of this restaurant in this little ...,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
257,Indian Palace Restaurant,The food served at this place was at best medi...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
258,Royal Indian Cuisine,Their food was worth the wait. Though it took ...,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Feature Extraction/Calculation

In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")


#  feature extraction
def extract_features(text):
    # Process the text with spaCy
    doc = nlp(text)

    words = [token.text for token in doc if not token.is_punct and not token.is_space]
    awl = sum(len(word) for word in words) / len(words) if words else 0

    sentences = [sent.text for sent in doc.sents]
    asl = len(words) / len(sentences) if words else 0

    nwo = len(words)

    nvb = len([token for token in doc if token.pos_ == "VERB"])

    naj = len([token for token in doc if token.pos_ == "ADJ"])

    npv = len([sent for sent in doc.sents if "by" in sent.text.lower()])

    nst = len(sentences)

    tokens = [token.text.lower() for token in doc if not token.is_punct and not token.is_space]
    cdv = len(set(tokens)) / len(tokens) if tokens else 0

    return awl, asl, nwo, nvb, naj, npv, nst, cdv

# Apply feature extraction to the 'Review' column
df[['AWL', 'ASL', 'NWO', 'NVB', 'NAJ', 'NPV', 'NST', 'CDV']] = df['Review'].apply(extract_features).apply(pd.Series)


/var/folders/4h/srzy3b3174x98fmcjyygj8lh0000gp/T/ipykernel_10629/2541522062.py:32: FutureWarning: Returning a DataFrame from Series.apply when the supplied function returns a Series is deprecated and will be removed in a future version.
  df[['AWL', 'ASL', 'NWO', 'NVB', 'NAJ', 'NPV', 'NST', 'CDV']] = df['Review'].apply(extract_features).apply(pd.Series)


In [ ]:
spell = SpellChecker()

def count_typos_and_tpr(text):
    # Process the text with spaCy
    doc = nlp(text)

    # Calculate Number of Typos (NTP)
    words = [token.text for token in doc if not token.is_punct and not token.is_space]
    misspelled = spell.unknown(words)
    ntp = len(misspelled)

    # Calculate Typo Ratio (TPR)
    tpr = ntp / len(words) if words else 0

    return ntp, tpr

# Apply the function to the 'Review' column
df[['NTP', 'TPR']] = df['Review'].apply(count_typos_and_tpr).apply(pd.Series)


/var/folders/4h/srzy3b3174x98fmcjyygj8lh0000gp/T/ipykernel_10629/1977517102.py:18: FutureWarning: Returning a DataFrame from Series.apply when the supplied function returns a Series is deprecated and will be removed in a future version.
  df[['NTP', 'TPR']] = df['Review'].apply(count_typos_and_tpr).apply(pd.Series)


In [ ]:
df.to_excel("group-3.xlsx", index=False)


In [ ]:
display(df)

,Restaurant,Review,Real=1/Fake=0,positive=1/negative=0,AWL,ASL,NOW,NVB,NAJ,NPV,NST,CDV,NTP,TPR,NWO
0,Tikka Shack,Great food and great atmosphere! The chicken t...,0,1,4.674699,16.600000,NaN,12.0,6.0,0.0,5.0,0.771084,3.0,0.036145,83.0
1,Tikka Shack,I had heard good things about Tikka Shak so I ...,0,0,4.073394,18.166667,NaN,15.0,8.0,0.0,6.0,0.733945,0.0,0.000000,109.0
2,Tikka Shack,I was driving by tikka shack one day and decid...,0,0,3.543860,16.285714,NaN,15.0,5.0,1.0,7.0,0.666667,3.0,0.026316,114.0
3,Tikka Shack,Tikka Shack had the most modern and up-to-date...,0,1,4.393443,15.250000,NaN,9.0,7.0,0.0,4.0,0.803279,0.0,0.000000,61.0
4,India Palace\nIndian\nRestaurant,Today is the third time I've come to India Pal...,0,1,4.440000,12.500000,NaN,10.0,9.0,0.0,6.0,0.760000,1.0,0.013333,75.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255,Tikka Shack,I usually don't eat outside very often and Tik...,0,1,3.962963,13.500000,NaN,4.0,4.0,0.0,4.0,0.759259,2.0,0.037037,54.0
256,Indian Palace Restaurant,The outside of this restaurant in this little ...,1,1,4.172043,18.600000,NaN,9.0,7.0,0.0,5.0,0.698925,3.0,0.032258,93.0
257,Indian Palace Restaurant,The food served at this place was at best medi...,0,0,4.458333,12.000000,NaN,7.0,9.0,0.0,6.0,0.666667,2.0,0.027778,72.0
258,Royal Indian Cuisine,Their food was worth the wait. Though it took ...,1,1,4.500000,18.666667,NaN,5.0,6.0,0.0,3.0,0.803571,5.0,0.089286,56.0




# Split and Shuffle

In [ ]:
from sklearn.model_selection import train_test_split

# 1. Split the data into a combined train + validation set and a test set
train_val_df, test_df = train_test_split(df, test_size=0.20, stratify=df['Real=1/Fake=0'], random_state=42)

# 2. Split the train + validation set into separate train and validation sets
train_df, val_df = train_test_split(train_val_df, test_size=0.25, stratify=train_val_df['Real=1/Fake=0'], random_state=42)

print(f"Train set size: {len(train_df)}")
print(f"Validation set size: {len(val_df)}")
print(f"Test set size: {len(test_df)}")


Train set size: 156
Validation set size: 52
Test set size: 52


In [ ]:
# train_test_split already shuffles, but just to be safe
train_df = train_df.sample(frac=1, random_state=42).reset_index(drop=True)
val_df = val_df.sample(frac=1, random_state=42).reset_index(drop=True)
test_df = test_df.sample(frac=1, random_state=42).reset_index(drop=True)


In [ ]:
# Extracting features and labels for model training
X_train = train_df[['AWL', 'ASL', 'NWO', 'NVB', 'NAJ', 'NPV', 'NST', 'CDV', 'NTP', 'TPR']]
y_train = train_df['Real=1/Fake=0']

X_val = val_df[['AWL', 'ASL', 'NWO', 'NVB', 'NAJ', 'NPV', 'NST', 'CDV', 'NTP', 'TPR']]
y_val = val_df['Real=1/Fake=0']

X_test = test_df[['AWL', 'ASL', 'NWO', 'NVB', 'NAJ', 'NPV', 'NST', 'CDV', 'NTP', 'TPR']]
y_test = test_df['Real=1/Fake=0']


# Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

# Model Hyperparameter Tuning
param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear']
}

logreg = LogisticRegression()

# Using GridSearchCV for 5-fold cross validation and F1-score optimization
grid_search = GridSearchCV(logreg, param_grid, cv=5, scoring='f1', return_train_score=True)

grid_search.fit(X_train, y_train)


GridSearchCV(cv=5, estimator=LogisticRegression(),
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
                         'penalty': ['l1', 'l2'], 'solver': ['liblinear']},
             return_train_score=True, scoring='f1')

In [ ]:
import numpy as np
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler

# Feature Selection
selector = SelectFromModel(grid_search.best_estimator_, threshold=-np.inf, max_features=10)
X_train_selected = selector.fit_transform(X_train, y_train)

best_logreg = grid_search.best_estimator_
best_logreg.fit(X_train_selected, y_train)

X_val_selected = selector.transform(X_val)

# Feature Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_selected)
X_val_scaled = scaler.transform(X_val_selected)

# Using the trained model to predict on validation data
y_val_pred = best_logreg.predict(X_val_selected)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

val_accuracy = accuracy_score(y_val, y_val_pred)
val_f1 = f1_score(y_val, y_val_pred)

print(f"Validation Accuracy: {val_accuracy:.4f}")
print(f"Validation F1 Score: {val_f1:.4f}")


Validation Accuracy: 0.5000
Validation F1 Score: 0.6667


In [ ]:
from sklearn.preprocessing import StandardScaler

# Hyperparameter Optimization with Scaled Data
scaler = StandardScaler()
X_train_selected_scaled = scaler.fit_transform(X_train_selected)
X_val_selected_scaled = scaler.transform(X_val_selected)

param_grid = {
    'C': np.logspace(-4, 4, 20),
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear']
}

grid_search = GridSearchCV(LogisticRegression(), param_grid, cv=5, scoring='f1', n_jobs=-1)
grid_search.fit(X_train_selected_scaled, y_train)

print("Best Parameters:", grid_search.best_params_)

y_val_pred = grid_search.predict(X_val_selected_scaled)
val_accuracy = accuracy_score(y_val, y_val_pred)
val_f1 = f1_score(y_val, y_val_pred)

print("Validation Accuracy:", val_accuracy)
print("Validation F1 Score:", val_f1)


Best Parameters: {'C': 11.288378916846883, 'penalty': 'l1', 'solver': 'liblinear'}
Validation Accuracy: 0.6730769230769231
Validation F1 Score: 0.6382978723404256


In [ ]:
# Testing the Model on Test Data

# Feature Selection

X_test_selected = selector.transform(X_test)

X_test_selected_scaled = scaler.transform(X_test_selected)

# Model Prediction
y_test_pred = grid_search.predict(X_test_selected_scaled)

test_accuracy = accuracy_score(y_test, y_test_pred)
test_f1 = f1_score(y_test, y_test_pred)

print("Test Accuracy:", test_accuracy)
print("Test F1 Score:", test_f1)


Test Accuracy: 0.75
Test F1 Score: 0.6976744186046512


# SVM Classifier

In [ ]:
from sklearn.svm import SVC

svc = SVC()  # You can change the kernel to 'poly', 'rbf', etc.
param_grid = {
    'C': [0.1, 1, 10, 100, 200],
    'kernel': ['linear'],
    #'degree': [2, 3, 4, 5],  # Only used when kernel is 'poly'
    'gamma': ['scale', 'auto'],
    'shrinking': [True, False],
    'tol': [1e-3, 1e-4, 1e-5]
}

grid_search = GridSearchCV(svc, param_grid, scoring='f1', cv=5, verbose=2, n_jobs=-1)
grid_search.fit(X_train, y_train)

selector = SelectFromModel(grid_search.best_estimator_, threshold=-np.inf, max_features=10)
selector = selector.fit(X_train, y_train)

Fitting 5 folds for each of 60 candidates, totalling 300 fits
[CV] END C=0.1, gamma=scale, kernel=linear, shrinking=True, tol=0.001; total time=   0.0s
[CV] END C=0.1, gamma=scale, kernel=linear, shrinking=True, tol=0.001; total time=   0.0s
[CV] END C=0.1, gamma=scale, kernel=linear, shrinking=True, tol=0.0001; total time=   0.0s
[CV] END C=0.1, gamma=scale, kernel=linear, shrinking=True, tol=0.0001; total time=   0.0s
[CV] END C=0.1, gamma=scale, kernel=linear, shrinking=True, tol=0.001; total time=   0.0s
[CV] END C=0.1, gamma=scale, kernel=linear, shrinking=True, tol=0.0001; total time=   0.0s
[CV] END C=0.1, gamma=scale, kernel=linear, shrinking=True, tol=0.001; total time=   0.0s
[CV] END C=0.1, gamma=scale, kernel=linear, shrinking=True, tol=1e-05; total time=   0.0s
[CV] END C=0.1, gamma=scale, kernel=linear, shrinking=True, tol=1e-05; total time=   0.0s
[CV] END C=0.1, gamma=scale, kernel=linear, shrinking=True, tol=1e-05; total time=   0.0s
[CV] END C=0.1, gamma=scale, kernel

In [ ]:
X_train_selected = selector.transform(X_train)
X_test_selected = selector.transform(X_test)
X_val_selected = selector.transform(X_val)

In [ ]:
# Scaling the features in all 3 sets
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_selected)
X_val_scaled = scaler.transform(X_val_selected)
X_test_scaled = scaler.transform(X_test_selected)

In [ ]:
# Testing model on validation set
best_svc = grid_search.best_estimator_
y_pred = best_svc.predict(X_val_scaled)

# Print results
print("Best hyperparameters:\n", grid_search.best_params_)
print("\nAccuracy:", accuracy_score(y_val, y_pred))
print("F1 Score:", f1_score(y_val, y_pred))

Best hyperparameters:
 {'C': 100, 'gamma': 'scale', 'kernel': 'linear', 'shrinking': True, 'tol': 0.001}

Accuracy: 0.6923076923076923
F1 Score: 0.5789473684210527


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(


In [ ]:
#Testing model on test set
best_svc = grid_search.best_estimator_
y_pred = best_svc.predict(X_test_selected)

# Print results
print("Best hyperparameters:\n", grid_search.best_params_)
print("\nAccuracy:", accuracy_score(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred))

Best hyperparameters:
 {'C': 100, 'gamma': 'scale', 'kernel': 'linear', 'shrinking': True, 'tol': 0.001}

Accuracy: 0.7692307692307693
F1 Score: 0.7142857142857143


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(


# Gradient Boost Classifier

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

unscaled_df = df[['AWL', 'ASL', 'NWO', 'NVB', 'NAJ', 'NPV', 'NST', 'CDV', 'NTP', 'TPR']]

df_scaled = scaler.fit_transform(unscaled_df.to_numpy())
df_scaled = pd.DataFrame(df_scaled, columns=['AWL', 'ASL', 'NWO', 'NVB', 'NAJ', 'NPV', 'NST', 'CDV', 'NTP', 'TPR'])

print("Scaled Dataset Using MinMaxScaler")
df_scaled.head()

Scaled Dataset Using MinMaxScaler


,AWL,ASL,NWO,NVB,NAJ,NPV,NST,CDV,NTP,TPR
0,0.582242,0.335260,0.243506,0.24,0.153846,0.0,0.16,0.549459,0.428571,0.156627
1,0.315203,0.380539,0.327922,0.30,0.205128,0.0,0.20,0.476363,0.000000,0.000000
2,0.080037,0.326177,0.344156,0.30,0.128205,0.5,0.24,0.343949,0.428571,0.114035
3,0.457336,0.296243,0.172078,0.18,0.179487,0.0,0.12,0.612822,0.000000,0.000000
4,0.478012,0.216763,0.217532,0.20,0.230769,0.0,0.20,0.527643,0.142857,0.057778


In [ ]:
from sklearn.model_selection import train_test_split

df_scaled['Real=1/Fake=0'] = df['Real=1/Fake=0'].to_numpy()

# 1. Split the data into a combined train + validation set and a test set
train_df, test_val_df = train_test_split(df_scaled, test_size=0.40, stratify=df_scaled['Real=1/Fake=0'], random_state=42)

# 2. Split the train + validation set into separate train and validation sets
test_df, val_df = train_test_split(test_val_df, test_size=0.50, stratify=test_val_df['Real=1/Fake=0'], random_state=42)

# Print results
print(f"Train set size: {len(train_df)}")
print(f"Validation set size: {len(val_df)}")
print(f"Test set size: {len(test_df)}")

Train set size: 156
Validation set size: 52
Test set size: 52


In [ ]:
train_df = train_df.sample(frac=1, random_state=42).reset_index(drop=True)
val_df = val_df.sample(frac=1, random_state=42).reset_index(drop=True)
test_df = test_df.sample(frac=1, random_state=42).reset_index(drop=True)

In [ ]:
# Extracting features and labels
X_train = train_df[['AWL', 'ASL', 'NWO', 'NVB', 'NAJ', 'NPV', 'NST', 'CDV', 'NTP', 'TPR']]
y_train = train_df['Real=1/Fake=0']

X_val = val_df[['AWL', 'ASL', 'NWO', 'NVB', 'NAJ', 'NPV', 'NST', 'CDV', 'NTP', 'TPR']]
y_val = val_df['Real=1/Fake=0']

X_test = test_df[['AWL', 'ASL', 'NWO', 'NVB', 'NAJ', 'NPV', 'NST', 'CDV', 'NTP', 'TPR']]
y_test = test_df['Real=1/Fake=0']

In [ ]:
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from sklearn.model_selection import cross_val_score, GridSearchCV
param_grid_gb = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.005,0.01, 0.05],
    'max_depth': [ 5, 6, 7],
    'subsample': [0.2, 0.3,0.4],
    'max_features': ['sqrt', 'log2', None]
}

grid_gb = GridSearchCV(GradientBoostingClassifier(), param_grid_gb, cv=5, scoring='accuracy', n_jobs=-1)
grid_gb.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=GradientBoostingClassifier(), n_jobs=-1,
             param_grid={'learning_rate': [0.005, 0.01, 0.05],
                         'max_depth': [5, 6, 7],
                         'max_features': ['sqrt', 'log2', None],
                         'n_estimators': [100, 200, 300],
                         'subsample': [0.2, 0.3, 0.4]},
             scoring='accuracy')

In [ ]:
from sklearn.feature_selection import SelectFromModel
# Feature Selection
selector = SelectFromModel(grid_gb.best_estimator_, threshold=-np.inf, max_features=10)
X_train_selected = selector.fit_transform(X_train, y_train)
X_val_selected = selector.transform(X_val)
X_test_selected = selector.transform(X_test)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

# Using the trained model to predict on validation data
y_val_pred = grid_gb.predict(X_val_selected)

val_accuracy = accuracy_score(y_val, y_val_pred)
val_f1 = f1_score(y_val, y_val_pred)

# Print results
print(f"Validation Accuracy: {val_accuracy:.4f}")
print(f"Validation F1 Score: {val_f1:.4f}")
print("Best hyperparameters:\n", grid_gb.best_params_)

Validation Accuracy: 0.6346
Validation F1 Score: 0.5957
Best hyperparameters:
 {'learning_rate': 0.005, 'max_depth': 6, 'max_features': 'log2', 'n_estimators': 100, 'subsample': 0.3}


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names
  warnings.warn(


In [ ]:
#Testing model on test set
test_predictions = grid_gb.predict(X_test_selected)
test_accuracy = accuracy_score(y_test, test_predictions)
gb_f1 = f1_score(y_test, test_predictions, average='macro')

# Print results
print("Gradient Boosting Test Accuracy:", test_accuracy)
print("Gradient Boosting Test Macro F1 Score:", gb_f1)
print("Best hyperparameters:\n", grid_gb.best_params_)

Gradient Boosting Test Accuracy: 0.6923076923076923
Gradient Boosting Test Macro F1 Score: 0.6904761904761905
Best hyperparameters:
 {'learning_rate': 0.005, 'max_depth': 6, 'max_features': 'log2', 'n_estimators': 100, 'subsample': 0.3}


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names
  warnings.warn(
